In [32]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
OHE = OneHotEncoder(sparse=False)
pd.set_option('display.max_rows', 30)

In [33]:
# 月日如果用起来要one-hot，之后产生的feature就太多了而且感觉没必要，可能考虑is_special更合适
# 但是前后依赖这里怎么办呢？
# train[['month','day']] = train.date.str.split('-', expand=True)
def get_x_y(df):
    data = df.drop(columns=['Unnamed: 0', 'route', 'calculated_day', 'date'])
    ohe = OHE.fit_transform(data[['hr', 'weekday']])
    ohe_df = pd.DataFrame(ohe, columns= OHE.get_feature_names(input_features=['hr', 'weekday']))
    data = pd.concat([data, ohe_df],axis = 1)
    data = data.drop(columns=['hr', 'weekday'])
    x = data.drop(columns=['label'])
    y = data.label
    return x, y

In [34]:
def mape_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def scoring(reg, x, y):
    pred = reg.predict(x)
    return -mape_error(pred, y)

In [47]:


import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}

file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '_' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        travel_seq = each_traj[4]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = math.floor(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        time=start_time_window
        tt = float(each_traj[-1])
        # 國慶不管，因道路狀態不一樣
        #if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        #    continue
        # 中秋節不管，因道路狀態不一樣
        if(time.month==9 and time.day in [15,16,17]):
            continue
        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

In [48]:
real_value=[]
real_value_dict={}
#fw = open('/Users/vayne/Desktop/dm_pro_engin/real_value.csv', 'w')
#fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
for route in travel_times.keys():
    if route not in real_value_dict.keys():
        real_value_dict[route] = {}
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        time_window_end = time_window_start + timedelta(minutes=20)
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        out_line =[str(route),time_window_start, str(avg_tt)]
        real_value.append(out_line)

In [49]:
real_value

[['B_3', datetime.datetime(2016, 7, 19, 0, 0), '70.85'],
 ['B_3', datetime.datetime(2016, 7, 19, 0, 20), '148.79'],
 ['B_3', datetime.datetime(2016, 7, 19, 1, 40), '93.72'],
 ['B_3', datetime.datetime(2016, 7, 19, 2, 0), '67.81'],
 ['B_3', datetime.datetime(2016, 7, 19, 2, 40), '167.55'],
 ['B_3', datetime.datetime(2016, 7, 19, 3, 40), '98.49'],
 ['B_3', datetime.datetime(2016, 7, 19, 4, 0), '96.67'],
 ['B_3', datetime.datetime(2016, 7, 19, 4, 20), '53.15'],
 ['B_3', datetime.datetime(2016, 7, 19, 4, 40), '55.25'],
 ['B_3', datetime.datetime(2016, 7, 19, 5, 0), '209.66'],
 ['B_3', datetime.datetime(2016, 7, 19, 6, 0), '145.94'],
 ['B_3', datetime.datetime(2016, 7, 19, 6, 20), '127.35'],
 ['B_3', datetime.datetime(2016, 7, 19, 6, 40), '97.2'],
 ['B_3', datetime.datetime(2016, 7, 19, 7, 0), '118.43'],
 ['B_3', datetime.datetime(2016, 7, 19, 7, 20), '82.63'],
 ['B_3', datetime.datetime(2016, 7, 19, 7, 40), '110.5'],
 ['B_3', datetime.datetime(2016, 7, 19, 8, 0), '125.76'],
 ['B_3', dateti

In [50]:
real_value_needed=[]
for i in real_value:
    if i[1].hour in [8,9,17,18] and i[1].month*31+i[1].day in [298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]:
        real_value_needed.append(i)

In [51]:
#convert to dataframe
feature=['route','time','label']
real_value_df=pd.DataFrame(real_value_needed,columns=feature)


In [52]:
real_value_df_A_2= real_value_df.loc[real_value_df['route'] == 'A_2']
real_value_df_A_2.to_csv('real_value_df_A_2.csv', index=False)

In [53]:
real_value_df_A_3= real_value_df.loc[real_value_df['route'] == 'A_3']
real_value_df_A_3.to_csv('real_value_df_A_3.csv', index=False)

In [54]:
real_value_df_B_1= real_value_df.loc[real_value_df['route'] == 'B_1']
real_value_df_B_1.to_csv('real_value_df_B_1.csv', index=False)

In [55]:
real_value_df_B_3= real_value_df.loc[real_value_df['route'] == 'B_3']
real_value_df_B_3.to_csv('real_value_df_B_3.csv', index=False)

In [56]:
real_value_df_C_1= real_value_df.loc[real_value_df['route'] == 'C_1']
real_value_df_C_1.to_csv('real_value_df_C_1.csv', index=False)

In [57]:
real_value_df_C_3= real_value_df.loc[real_value_df['route'] == 'C_3']
real_value_df_C_3.to_csv('real_value_df_C_3.csv', index=False)

In [82]:
real_value_df_A_2


,route,time,label
332,A_2,2016-09-19 08:00:00,102.49
333,A_2,2016-09-19 08:20:00,61.68
334,A_2,2016-09-19 08:40:00,76.21
335,A_2,2016-09-19 09:00:00,66.97
336,A_2,2016-09-19 09:20:00,69.78
...,...,...,...
505,A_2,2016-10-17 17:20:00,58.04
506,A_2,2016-10-17 17:40:00,53.69
507,A_2,2016-10-17 18:00:00,65.4
508,A_2,2016-10-17 18:20:00,48.66


## B1

In [59]:
#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_1_travel_times=travel_times['B_1']
B_1_travel_times_need={}
for i in B_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_1_travel_times_need:
            B_1_travel_times_need[i]=[]
        if i in B_1_travel_times_need:
            B_1_travel_times_need[i].extend(B_1_travel_times[i])
        
B_1_ratio_8_9={}
B_1_ratio_9_10={}
B_1_ratio_17_18={}
B_1_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_1_ratio_8_9[i]={}
    B_1_ratio_9_10[i]={}
    B_1_ratio_17_18[i]={}
    B_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_1_ratio_18_19[i][j]=[]

for i in B_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    for k in [480,500,520]:
        if B_1_ratio_8_9[i][k]==[] and B_1_ratio_8_9[i][k-20]!=[] and B_1_ratio_8_9[i][k+20]!=[]:
            B_1_ratio_8_9[i][k]=[np.mean([np.mean(B_1_ratio_8_9[i][k-20]),np.mean(B_1_ratio_8_9[i][k+20])])]
            
for i in B_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_1_ratio_9_10[i][k]==[] and B_1_ratio_9_10[i][k-20]!=[] and B_1_ratio_9_10[i][k+20]!=[]:
            B_1_ratio_9_10[i][k]=[np.mean([np.mean(B_1_ratio_9_10[i][k-20]),np.mean(B_1_ratio_9_10[i][k+20])])]
            
for i in B_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_1_ratio_17_18[i][k]==[] and B_1_ratio_17_18[i][k-20]!=[] and B_1_ratio_17_18[i][k+20]!=[]:
            B_1_ratio_17_18[i][k]=[np.mean([np.mean(B_1_ratio_17_18[i][k-20]),np.mean(B_1_ratio_17_18[i][k+20])])]
            
for i in B_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_1_ratio_18_19[i][k]==[] and B_1_ratio_18_19[i][k-20]!=[] and B_1_ratio_18_19[i][k+20]!=[]:
            B_1_ratio_18_19[i][k]=[np.mean([np.mean(B_1_ratio_18_19[i][k-20]),np.mean(B_1_ratio_18_19[i][k+20])])]
            
B_1_ratio_8_9_key=list(B_1_ratio_8_9.keys())
for i in B_1_ratio_8_9_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_1_ratio_8_9[i]
        
B_1_ratio_9_10_key=list(B_1_ratio_9_10.keys())
for i in B_1_ratio_9_10_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_1_ratio_9_10[i]

B_1_ratio_17_18_key=list(B_1_ratio_17_18.keys())
for i in B_1_ratio_17_18_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_1_ratio_17_18[i]

B_1_ratio_18_19_key=list(B_1_ratio_18_19.keys())
for i in  B_1_ratio_18_19_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    del B_1_ratio_8_9[i][8*60-20]
    del B_1_ratio_8_9[i][9*60]
    
for i in B_1_ratio_9_10:
    del B_1_ratio_9_10[i][9*60-20]
    del B_1_ratio_9_10[i][10*60]
    
for i in B_1_ratio_17_18:
    del B_1_ratio_17_18[i][17*60-20]
    del B_1_ratio_17_18[i][18*60]
    
for i in B_1_ratio_18_19:
    del B_1_ratio_18_19[i][18*60-20]
    del B_1_ratio_18_19[i][19*60]
    
for i in B_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_1_ratio_8_9[i]: 
        if B_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_8_9[i][j]))
            temp2.append(np.mean(B_1_ratio_8_9[i][j]))
        if B_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_1_ratio_9_10[i]: 
        if B_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_9_10[i][j]))
            temp2.append(np.mean(B_1_ratio_9_10[i][j]))
        if B_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_1_ratio_17_18[i]: 
        if B_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_17_18[i][j]))
            temp2.append(np.mean(B_1_ratio_17_18[i][j]))
        if B_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_1_ratio_18_19[i]: 
        if B_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_18_19[i][j]))
            temp2.append(np.mean(B_1_ratio_18_19[i][j]))
        if B_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_1_ratio_8_9_480=[]
B_1_ratio_8_9_500=[]
B_1_ratio_8_9_520=[]

for i in B_1_ratio_8_9:
    for j in B_1_ratio_8_9[i]:
        if j==480: B_1_ratio_8_9_480.extend(B_1_ratio_8_9[i][j])
        if j==500: B_1_ratio_8_9_500.extend(B_1_ratio_8_9[i][j])
        if j==520: B_1_ratio_8_9_520.extend(B_1_ratio_8_9[i][j])
    
    
B_1_ratio_9_10_540=[]
B_1_ratio_9_10_560=[]
B_1_ratio_9_10_580=[]

for i in B_1_ratio_9_10:
    for j in B_1_ratio_9_10[i]:
        if j==540: B_1_ratio_9_10_540.extend(B_1_ratio_9_10[i][j])
        if j==560: B_1_ratio_9_10_560.extend(B_1_ratio_9_10[i][j])
        if j==580: B_1_ratio_9_10_580.extend(B_1_ratio_9_10[i][j])
            
B_1_ratio_17_18_1020=[]
B_1_ratio_17_18_1040=[]
B_1_ratio_17_18_1060=[]

for i in B_1_ratio_17_18:
    for j in B_1_ratio_17_18[i]:
        if j==1020: B_1_ratio_17_18_1020.extend(B_1_ratio_17_18[i][j])
        if j==1040: B_1_ratio_17_18_1040.extend(B_1_ratio_17_18[i][j])
        if j==1060: B_1_ratio_17_18_1060.extend(B_1_ratio_17_18[i][j])
    
B_1_ratio_18_19_1080=[]
B_1_ratio_18_19_1100=[]
B_1_ratio_18_19_1120=[]

for i in B_1_ratio_18_19:
    for j in B_1_ratio_18_19[i]:
        if j==1080: B_1_ratio_18_19_1080.extend(B_1_ratio_18_19[i][j])
        if j==1100: B_1_ratio_18_19_1100.extend(B_1_ratio_18_19[i][j])
        if j==1120: B_1_ratio_18_19_1120.extend(B_1_ratio_18_19[i][j])
            
B_1_ratio_8_9_480.sort()
B_1_ratio_8_9_480_used=B_1_ratio_8_9_480[5:len(B_1_ratio_8_9_480)-5]

B_1_ratio_8_9_500.sort()
B_1_ratio_8_9_500_used=B_1_ratio_8_9_500[5:len(B_1_ratio_8_9_500)-5]

B_1_ratio_8_9_520.sort()
B_1_ratio_8_9_520_used=B_1_ratio_8_9_520[5:len(B_1_ratio_8_9_520)-5]

B_1_ratio_9_10_540.sort()
B_1_ratio_9_10_540_used=B_1_ratio_9_10_540[5:len(B_1_ratio_9_10_540)-5]

B_1_ratio_9_10_560.sort()
B_1_ratio_9_10_560_used=B_1_ratio_9_10_560[5:len(B_1_ratio_9_10_560)-5]

B_1_ratio_9_10_580.sort()
B_1_ratio_9_10_580_used=B_1_ratio_9_10_580[5:len(B_1_ratio_9_10_580)-5]

B_1_ratio_17_18_1020.sort()
B_1_ratio_17_18_1020_used=B_1_ratio_17_18_1020[5:len(B_1_ratio_17_18_1020)-5]

B_1_ratio_17_18_1040.sort()
B_1_ratio_17_18_1040_used=B_1_ratio_17_18_1040[5:len(B_1_ratio_17_18_1040)-5]

B_1_ratio_17_18_1060.sort()
B_1_ratio_17_18_1060_used=B_1_ratio_17_18_1060[5:len(B_1_ratio_17_18_1060)-5]

B_1_ratio_18_19_1080.sort()
B_1_ratio_18_19_1080_used=B_1_ratio_18_19_1080[5:len(B_1_ratio_18_19_1080)-5]

B_1_ratio_18_19_1100.sort()
B_1_ratio_18_19_1100_used=B_1_ratio_18_19_1100[5:len(B_1_ratio_18_19_1100)-5]

B_1_ratio_18_19_1120.sort()
B_1_ratio_18_19_1120_used=B_1_ratio_18_19_1120[5:len(B_1_ratio_18_19_1120)-5]

names=locals()
B_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_1_ratio.append(np.mean(names['B_1_ratio_'+str(i)+'_used']))
    
B_1_ratio_8=B_1_ratio[0:3]
B_1_ratio_9=B_1_ratio[3:6]
B_1_ratio_17=B_1_ratio[6:9]
B_1_ratio_18=B_1_ratio[9:12]



In [60]:
print(B_1_ratio_8)
print(B_1_ratio_9)
print(B_1_ratio_17)
print(B_1_ratio_18)

[1.0031420374343412, 0.9934482062307293, 0.9873723329424676]
[1.0143332259275053, 0.9560381921561566, 0.9965976774126419]
[0.9289831850456122, 1.000556880529891, 1.0657150770402124]
[1.0174539659659674, 1.0053514813377316, 0.9790868479720023]


In [61]:
#B_1
file_path = '/Users/vayne/Desktop/validation/B_1.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('/Users/vayne/Desktop/validation/5min_shifted_with_10.1_B_1.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/validation/-5min_shifted_with_10.1_B_1.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = GradientBoostingRegressor(n_estimators=5, learning_rate=0.1, 
                                 max_depth=3, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.2405223397424579
0.1299667129187069
          label           0
0    100.320000  120.109011
1    212.310000  141.790574
2    129.945000  122.719566
3    118.616667  122.719566
4    118.785000  122.719566
..          ...         ...
462  121.953333  132.897367
463  110.900000  124.382294
464  144.060000  126.992849
465  130.540000  124.525558
466  215.270000  138.686647

[467 rows x 2 columns]
         label           0
0   132.930000  124.525558
1   163.843333  119.589282
2    96.410000  125.387848
3   128.306667  124.382294
4   133.493333  126.992849
..         ...         ...
55  148.810000  124.525558
56  203.920000  124.525558
57  111.363333  141.422004
58  117.240000  124.525558
59  120.330000  124.525558

[60 rows x 2 columns]


In [62]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])

In [64]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_B_1.loc[real_value_df_B_1['route'] == 'B_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_B_1_time=list(real_value_df_B_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_B_1_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_B_1=real_value_df.loc[real_value_df['route'] =='B_1']

real_value=list(real_value_df_B_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.14426880667049694

## A2

In [246]:
#A2

#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    #國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_2_travel_times=travel_times['A_2']
A_2_travel_times_need={}
for i in A_2_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_2_travel_times_need:
            A_2_travel_times_need[i]=[]
        if i in A_2_travel_times_need:
            A_2_travel_times_need[i].extend(A_2_travel_times[i])
        
A_2_ratio_8_9={}
A_2_ratio_9_10={}
A_2_ratio_17_18={}
A_2_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]


for i in date_key:
    A_2_ratio_8_9[i]={}
    A_2_ratio_9_10[i]={}
    A_2_ratio_17_18[i]={}
    A_2_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_2_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_2_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_2_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_2_ratio_18_19[i][j]=[]

for i in A_2_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_2_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_2_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_2_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_2_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_2_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_2_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_2_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    for k in [480,500,520]:
        if A_2_ratio_8_9[i][k]==[] and A_2_ratio_8_9[i][k-20]!=[] and A_2_ratio_8_9[i][k+20]!=[]:
            A_2_ratio_8_9[i][k]=[np.mean([np.mean(A_2_ratio_8_9[i][k-20]),np.mean(A_2_ratio_8_9[i][k+20])])]
            
for i in A_2_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_2_ratio_9_10[i][k]==[] and A_2_ratio_9_10[i][k-20]!=[] and A_2_ratio_9_10[i][k+20]!=[]:
            A_2_ratio_9_10[i][k]=[np.mean([np.mean(A_2_ratio_9_10[i][k-20]),np.mean(A_2_ratio_9_10[i][k+20])])]
            
for i in A_2_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_2_ratio_17_18[i][k]==[] and A_2_ratio_17_18[i][k-20]!=[] and A_2_ratio_17_18[i][k+20]!=[]:
            A_2_ratio_17_18[i][k]=[np.mean([np.mean(A_2_ratio_17_18[i][k-20]),np.mean(A_2_ratio_17_18[i][k+20])])]
            
for i in A_2_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_2_ratio_18_19[i][k]==[] and A_2_ratio_18_19[i][k-20]!=[] and A_2_ratio_18_19[i][k+20]!=[]:
            A_2_ratio_18_19[i][k]=[np.mean([np.mean(A_2_ratio_18_19[i][k-20]),np.mean(A_2_ratio_18_19[i][k+20])])]
            
A_2_ratio_8_9_key=list(A_2_ratio_8_9.keys())
for i in A_2_ratio_8_9_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_2_ratio_8_9[i]
        
A_2_ratio_9_10_key=list(A_2_ratio_9_10.keys())
for i in A_2_ratio_9_10_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_2_ratio_9_10[i]

A_2_ratio_17_18_key=list(A_2_ratio_17_18.keys())
for i in A_2_ratio_17_18_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_2_ratio_17_18[i]

A_2_ratio_18_19_key=list(A_2_ratio_18_19.keys())
for i in  A_2_ratio_18_19_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    del A_2_ratio_8_9[i][8*60-20]
    del A_2_ratio_8_9[i][9*60]
    
for i in A_2_ratio_9_10:
    del A_2_ratio_9_10[i][9*60-20]
    del A_2_ratio_9_10[i][10*60]
    
for i in A_2_ratio_17_18:
    del A_2_ratio_17_18[i][17*60-20]
    del A_2_ratio_17_18[i][18*60]
    
for i in A_2_ratio_18_19:
    del A_2_ratio_18_19[i][18*60-20]
    del A_2_ratio_18_19[i][19*60]
    
for i in A_2_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_2_ratio_8_9[i]: 
        if A_2_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_8_9[i][j]))
            temp2.append(np.mean(A_2_ratio_8_9[i][j]))
        if A_2_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_2_ratio_9_10[i]: 
        if A_2_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_9_10[i][j]))
            temp2.append(np.mean(A_2_ratio_9_10[i][j]))
        if A_2_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_2_ratio_17_18[i]: 
        if A_2_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_17_18[i][j]))
            temp2.append(np.mean(A_2_ratio_17_18[i][j]))
        if A_2_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_2_ratio_18_19[i]: 
        if A_2_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_18_19[i][j]))
            temp2.append(np.mean(A_2_ratio_18_19[i][j]))
        if A_2_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_2_ratio_8_9_480=[]
A_2_ratio_8_9_500=[]
A_2_ratio_8_9_520=[]

for i in A_2_ratio_8_9:
    for j in A_2_ratio_8_9[i]:
        if j==480: A_2_ratio_8_9_480.extend(A_2_ratio_8_9[i][j])
        if j==500: A_2_ratio_8_9_500.extend(A_2_ratio_8_9[i][j])
        if j==520: A_2_ratio_8_9_520.extend(A_2_ratio_8_9[i][j])
    
    
A_2_ratio_9_10_540=[]
A_2_ratio_9_10_560=[]
A_2_ratio_9_10_580=[]

for i in A_2_ratio_9_10:
    for j in A_2_ratio_9_10[i]:
        if j==540: A_2_ratio_9_10_540.extend(A_2_ratio_9_10[i][j])
        if j==560: A_2_ratio_9_10_560.extend(A_2_ratio_9_10[i][j])
        if j==580: A_2_ratio_9_10_580.extend(A_2_ratio_9_10[i][j])
            
A_2_ratio_17_18_1020=[]
A_2_ratio_17_18_1040=[]
A_2_ratio_17_18_1060=[]

for i in A_2_ratio_17_18:
    for j in A_2_ratio_17_18[i]:
        if j==1020: A_2_ratio_17_18_1020.extend(A_2_ratio_17_18[i][j])
        if j==1040: A_2_ratio_17_18_1040.extend(A_2_ratio_17_18[i][j])
        if j==1060: A_2_ratio_17_18_1060.extend(A_2_ratio_17_18[i][j])
    
A_2_ratio_18_19_1080=[]
A_2_ratio_18_19_1100=[]
A_2_ratio_18_19_1120=[]

for i in A_2_ratio_18_19:
    for j in A_2_ratio_18_19[i]:
        if j==1080: A_2_ratio_18_19_1080.extend(A_2_ratio_18_19[i][j])
        if j==1100: A_2_ratio_18_19_1100.extend(A_2_ratio_18_19[i][j])
        if j==1120: A_2_ratio_18_19_1120.extend(A_2_ratio_18_19[i][j])
            
A_2_ratio_8_9_480.sort()
A_2_ratio_8_9_480_used=A_2_ratio_8_9_480[5:len(A_2_ratio_8_9_480)-5]

A_2_ratio_8_9_500.sort()
A_2_ratio_8_9_500_used=A_2_ratio_8_9_500[5:len(A_2_ratio_8_9_500)-5]

A_2_ratio_8_9_520.sort()
A_2_ratio_8_9_520_used=A_2_ratio_8_9_520[5:len(A_2_ratio_8_9_520)-5]

A_2_ratio_9_10_540.sort()
A_2_ratio_9_10_540_used=A_2_ratio_9_10_540[5:len(A_2_ratio_9_10_540)-5]

A_2_ratio_9_10_560.sort()
A_2_ratio_9_10_560_used=A_2_ratio_9_10_560[5:len(A_2_ratio_9_10_560)-5]

A_2_ratio_9_10_580.sort()
A_2_ratio_9_10_580_used=A_2_ratio_9_10_580[5:len(A_2_ratio_9_10_580)-5]

A_2_ratio_17_18_1020.sort()
A_2_ratio_17_18_1020_used=A_2_ratio_17_18_1020[5:len(A_2_ratio_17_18_1020)-5]

A_2_ratio_17_18_1040.sort()
A_2_ratio_17_18_1040_used=A_2_ratio_17_18_1040[5:len(A_2_ratio_17_18_1040)-5]

A_2_ratio_17_18_1060.sort()
A_2_ratio_17_18_1060_used=A_2_ratio_17_18_1060[5:len(A_2_ratio_17_18_1060)-5]

A_2_ratio_18_19_1080.sort()
A_2_ratio_18_19_1080_used=A_2_ratio_18_19_1080[5:len(A_2_ratio_18_19_1080)-5]

A_2_ratio_18_19_1100.sort()
A_2_ratio_18_19_1100_used=A_2_ratio_18_19_1100[5:len(A_2_ratio_18_19_1100)-5]

A_2_ratio_18_19_1120.sort()
A_2_ratio_18_19_1120_used=A_2_ratio_18_19_1120[5:len(A_2_ratio_18_19_1120)-5]

names=locals()
A_2_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_2_ratio.append(np.mean(names['A_2_ratio_'+str(i)+'_used']))
    
A_2_ratio_8=A_2_ratio[0:3]
A_2_ratio_9=A_2_ratio[3:6]
A_2_ratio_17=A_2_ratio[6:9]
A_2_ratio_18=A_2_ratio[9:12]





In [247]:
print(A_2_ratio_8)
print(A_2_ratio_9)
print(A_2_ratio_17)
print(A_2_ratio_18)


[0.9594377262857952, 1.0144147364368832, 1.004964586594884]
[1.000155698203997, 1.0194005415982434, 0.9602155822869323]
[1.0152742931390726, 0.9915728865263712, 0.9819021063140361]
[1.0065583872672326, 0.969453070970589, 1.0087499022367605]


In [248]:
#A_2
file_path = '/Users/vayne/Desktop/validation/A_2.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = GradientBoostingRegressor(n_estimators=24, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.09230943513239677
0.1381704938850841
          label           0
0     75.086667   74.010723
1    148.053333  129.465133
2    109.863333   99.966562
3     74.633333   85.159091
4     88.636667   89.937124
..          ...         ...
286   63.916667   66.903128
287   68.750000   67.771911
288   58.106667   61.798314
289   58.666667   63.324965
290   63.286667   65.068688

[291 rows x 2 columns]
        label          0
0   80.126667  71.999733
1   68.713333  75.483470
2   62.150000  94.890671
3   68.540000  73.725799
4   73.396667  77.115529
..        ...        ...
55  67.673333  73.960218
56  67.823333  80.144143
57  69.840000  76.180705
58  62.610000  70.323975
59  58.156667  69.260082

[60 rows x 2 columns]


In [249]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_2_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_2_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_2_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_2_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])

In [250]:
final_result

[['9-19', 8, 69.07926037885971],
 ['9-19', 8, 73.03759044555437],
 ['9-19', 8, 72.35718217759738],
 ['9-20', 8, 72.4216887015429],
 ['9-20', 8, 76.57154419067098],
 ['9-20', 8, 75.85821408983196],
 ['9-21', 8, 91.04168964667346],
 ['9-21', 8, 96.25849503044094],
 ['9-21', 8, 95.36176397071861],
 ['9-22', 8, 70.73531254498269],
 ['9-22', 8, 74.78853652115508],
 ['9-22', 8, 74.09181667748321],
 ['9-23', 8, 73.98754758071246],
 ['9-23', 8, 78.22712878849507],
 ['9-23', 8, 77.49837548651004],
 ['9-24', 8, 74.02951285291556],
 ['9-24', 8, 78.27149872452647],
 ['9-24', 8, 77.54233207824653],
 ['9-25', 8, 70.1826175954188],
 ['9-25', 8, 74.20417144332735],
 ['9-25', 8, 73.51289546531471],
 ['10-10', 8, 94.58319253743538],
 ['10-10', 8, 100.00293057127621],
 ['10-10', 8, 99.07131685886402],
 ['10-11', 8, 65.33311894389128],
 ['10-11', 8, 69.07679030991657],
 ['10-11', 8, 68.43328031781417],
 ['10-12', 8, 81.77445665224921],
 ['10-12', 8, 86.46023771995266],
 ['10-12', 8, 85.65478589390925],
 [

In [251]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_A_2.loc[real_value_df_A_2['route'] == 'A_2']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_A_2_time=list(real_value_df_A_2['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_A_2_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_A_2=real_value_df.loc[real_value_df['route'] =='A_2']

real_value=list(real_value_df_A_2['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.18032249079453194

In [86]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[102.49, 98.89164650253157],
 [61.68, 97.93601021886568],
 [76.21, 97.33703909514155],
 [66.97, 79.28916357350467],
 [69.78, 74.73231346737417],
 [141.19, 77.90279785924447],
 [63.53, 66.21835774043566],
 [63.54, 71.32016437017597],
 [64.11, 75.96467121992076],
 [58.54, 72.81641814359982],
 [54.96, 71.9502761747792],
 [78.85, 70.07058766844804],
 [58.94, 68.30917343956185],
 [70.15, 67.64906991257283],
 [77.05, 67.23533201030509],
 [83.06, 67.33286627542877],
 [70.83, 63.46315993719988],
 [85.52, 66.15555561858747],
 [65.65, 74.07066766553359],
 [59.27, 79.7774570855701],
 [60.11, 84.97271917114229],
 [77.01, 75.78747362726057],
 [64.21, 74.88599133394035],
 [62.39, 72.9296077774174],
 [55.45, 85.49944702902809],
 [66.75, 84.67322583943329],
 [64.25, 84.1553691581467],
 [108.22, 68.5749329522517],
 [61.66, 64.63384344622266],
 [94.26, 67.3759048427603],
 [69.95, 61.98823662952436],
 [56.71, 66.7641327313587],
 [70.97, 71.1119420013796],
 [78.48, 61.89958832805704],
 [56.71, 61.1633006

## A3 

In [252]:
#A3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_3_travel_times=travel_times['A_3']
A_3_travel_times_need={}
for i in A_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_3_travel_times_need:
            A_3_travel_times_need[i]=[]
        if i in A_3_travel_times_need:
            A_3_travel_times_need[i].extend(A_3_travel_times[i])
        
A_3_ratio_8_9={}
A_3_ratio_9_10={}
A_3_ratio_17_18={}
A_3_ratio_18_19={}

#date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]
date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    A_3_ratio_8_9[i]={}
    A_3_ratio_9_10[i]={}
    A_3_ratio_17_18[i]={}
    A_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_3_ratio_18_19[i][j]=[]

for i in A_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    for k in [480,500,520]:
        if A_3_ratio_8_9[i][k]==[] and A_3_ratio_8_9[i][k-20]!=[] and A_3_ratio_8_9[i][k+20]!=[]:
            A_3_ratio_8_9[i][k]=[np.mean([np.mean(A_3_ratio_8_9[i][k-20]),np.mean(A_3_ratio_8_9[i][k+20])])]
            
for i in A_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_3_ratio_9_10[i][k]==[] and A_3_ratio_9_10[i][k-20]!=[] and A_3_ratio_9_10[i][k+20]!=[]:
            A_3_ratio_9_10[i][k]=[np.mean([np.mean(A_3_ratio_9_10[i][k-20]),np.mean(A_3_ratio_9_10[i][k+20])])]
            
for i in A_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_3_ratio_17_18[i][k]==[] and A_3_ratio_17_18[i][k-20]!=[] and A_3_ratio_17_18[i][k+20]!=[]:
            A_3_ratio_17_18[i][k]=[np.mean([np.mean(A_3_ratio_17_18[i][k-20]),np.mean(A_3_ratio_17_18[i][k+20])])]
            
for i in A_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_3_ratio_18_19[i][k]==[] and A_3_ratio_18_19[i][k-20]!=[] and A_3_ratio_18_19[i][k+20]!=[]:
            A_3_ratio_18_19[i][k]=[np.mean([np.mean(A_3_ratio_18_19[i][k-20]),np.mean(A_3_ratio_18_19[i][k+20])])]
            
A_3_ratio_8_9_key=list(A_3_ratio_8_9.keys())
for i in A_3_ratio_8_9_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_3_ratio_8_9[i]
        
A_3_ratio_9_10_key=list(A_3_ratio_9_10.keys())
for i in A_3_ratio_9_10_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_3_ratio_9_10[i]

A_3_ratio_17_18_key=list(A_3_ratio_17_18.keys())
for i in A_3_ratio_17_18_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_3_ratio_17_18[i]

A_3_ratio_18_19_key=list(A_3_ratio_18_19.keys())
for i in  A_3_ratio_18_19_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    del A_3_ratio_8_9[i][8*60-20]
    del A_3_ratio_8_9[i][9*60]
    
for i in A_3_ratio_9_10:
    del A_3_ratio_9_10[i][9*60-20]
    del A_3_ratio_9_10[i][10*60]
    
for i in A_3_ratio_17_18:
    del A_3_ratio_17_18[i][17*60-20]
    del A_3_ratio_17_18[i][18*60]
    
for i in A_3_ratio_18_19:
    del A_3_ratio_18_19[i][18*60-20]
    del A_3_ratio_18_19[i][19*60]
    
for i in A_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_3_ratio_8_9[i]: 
        if A_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_8_9[i][j]))
            temp2.append(np.mean(A_3_ratio_8_9[i][j]))
        if A_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_3_ratio_9_10[i]: 
        if A_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_9_10[i][j]))
            temp2.append(np.mean(A_3_ratio_9_10[i][j]))
        if A_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_3_ratio_17_18[i]: 
        if A_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_17_18[i][j]))
            temp2.append(np.mean(A_3_ratio_17_18[i][j]))
        if A_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_3_ratio_18_19[i]: 
        if A_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_18_19[i][j]))
            temp2.append(np.mean(A_3_ratio_18_19[i][j]))
        if A_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_3_ratio_8_9_480=[]
A_3_ratio_8_9_500=[]
A_3_ratio_8_9_520=[]

for i in A_3_ratio_8_9:
    for j in A_3_ratio_8_9[i]:
        if j==480: A_3_ratio_8_9_480.extend(A_3_ratio_8_9[i][j])
        if j==500: A_3_ratio_8_9_500.extend(A_3_ratio_8_9[i][j])
        if j==520: A_3_ratio_8_9_520.extend(A_3_ratio_8_9[i][j])
    
    
A_3_ratio_9_10_540=[]
A_3_ratio_9_10_560=[]
A_3_ratio_9_10_580=[]

for i in A_3_ratio_9_10:
    for j in A_3_ratio_9_10[i]:
        if j==540: A_3_ratio_9_10_540.extend(A_3_ratio_9_10[i][j])
        if j==560: A_3_ratio_9_10_560.extend(A_3_ratio_9_10[i][j])
        if j==580: A_3_ratio_9_10_580.extend(A_3_ratio_9_10[i][j])
            
A_3_ratio_17_18_1020=[]
A_3_ratio_17_18_1040=[]
A_3_ratio_17_18_1060=[]

for i in A_3_ratio_17_18:
    for j in A_3_ratio_17_18[i]:
        if j==1020: A_3_ratio_17_18_1020.extend(A_3_ratio_17_18[i][j])
        if j==1040: A_3_ratio_17_18_1040.extend(A_3_ratio_17_18[i][j])
        if j==1060: A_3_ratio_17_18_1060.extend(A_3_ratio_17_18[i][j])
    
A_3_ratio_18_19_1080=[]
A_3_ratio_18_19_1100=[]
A_3_ratio_18_19_1120=[]

for i in A_3_ratio_18_19:
    for j in A_3_ratio_18_19[i]:
        if j==1080: A_3_ratio_18_19_1080.extend(A_3_ratio_18_19[i][j])
        if j==1100: A_3_ratio_18_19_1100.extend(A_3_ratio_18_19[i][j])
        if j==1120: A_3_ratio_18_19_1120.extend(A_3_ratio_18_19[i][j])
            
A_3_ratio_8_9_480.sort()
A_3_ratio_8_9_480_used=A_3_ratio_8_9_480[5:len(A_3_ratio_8_9_480)-5]

A_3_ratio_8_9_500.sort()
A_3_ratio_8_9_500_used=A_3_ratio_8_9_500[5:len(A_3_ratio_8_9_500)-5]

A_3_ratio_8_9_520.sort()
A_3_ratio_8_9_520_used=A_3_ratio_8_9_520[5:len(A_3_ratio_8_9_520)-5]

A_3_ratio_9_10_540.sort()
A_3_ratio_9_10_540_used=A_3_ratio_9_10_540[5:len(A_3_ratio_9_10_540)-5]

A_3_ratio_9_10_560.sort()
A_3_ratio_9_10_560_used=A_3_ratio_9_10_560[5:len(A_3_ratio_9_10_560)-5]

A_3_ratio_9_10_580.sort()
A_3_ratio_9_10_580_used=A_3_ratio_9_10_580[5:len(A_3_ratio_9_10_580)-5]

A_3_ratio_17_18_1020.sort()
A_3_ratio_17_18_1020_used=A_3_ratio_17_18_1020[5:len(A_3_ratio_17_18_1020)-5]

A_3_ratio_17_18_1040.sort()
A_3_ratio_17_18_1040_used=A_3_ratio_17_18_1040[5:len(A_3_ratio_17_18_1040)-5]

A_3_ratio_17_18_1060.sort()
A_3_ratio_17_18_1060_used=A_3_ratio_17_18_1060[5:len(A_3_ratio_17_18_1060)-5]

A_3_ratio_18_19_1080.sort()
A_3_ratio_18_19_1080_used=A_3_ratio_18_19_1080[5:len(A_3_ratio_18_19_1080)-5]

A_3_ratio_18_19_1100.sort()
A_3_ratio_18_19_1100_used=A_3_ratio_18_19_1100[5:len(A_3_ratio_18_19_1100)-5]

A_3_ratio_18_19_1120.sort()
A_3_ratio_18_19_1120_used=A_3_ratio_18_19_1120[5:len(A_3_ratio_18_19_1120)-5]

names=locals()
A_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_3_ratio.append(np.mean(names['A_3_ratio_'+str(i)+'_used']))
    
A_3_ratio_8=A_3_ratio[0:3]
A_3_ratio_9=A_3_ratio[3:6]
A_3_ratio_17=A_3_ratio[6:9]
A_3_ratio_18=A_3_ratio[9:12]

In [253]:

print(A_3_ratio_8)
print(A_3_ratio_9)
print(A_3_ratio_17)
print(A_3_ratio_18)

[0.9702602614629005, 0.978968720287374, 1.0444125878050396]
[1.057196991804728, 1.0448782719176082, 0.8779080952581202]
[1.0131716009078262, 1.008282644002323, 0.977339452004021]
[0.9618902905147334, 1.0074499810834463, 1.0076270414107436]


In [254]:
#A_3
file_path = '/Users/vayne/Desktop/validation/A_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = GradientBoostingRegressor(n_estimators=15, learning_rate=0.1, 
                                 max_depth=4, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.14110124946902966
0.1497435854873128
          label           0
0    146.513333  140.549010
1    203.340000  170.550275
2    168.586667  167.847055
3    139.636667  135.038876
4    108.560000  136.468762
..          ...         ...
282  115.176667  132.764129
283  109.960000  122.977151
284  123.546667  122.977151
285  105.820000  123.718420
286  120.643333  121.265205

[287 rows x 2 columns]
         label           0
0   124.720000  138.905539
1   130.993333  122.977151
2   113.410000  122.977151
3   124.660000  122.977151
4   124.023333  172.966660
..         ...         ...
55  110.613333  128.440411
56  140.670000  122.977151
57  257.630000  141.871483
58  137.190000  155.652048
59  129.810000  122.977151

[60 rows x 2 columns]


In [255]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])


In [258]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_A_3.loc[real_value_df_A_3['route'] == 'A_3']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_A_3_time=list(real_value_df_A_3['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_A_3_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_A_3=real_value_df.loc[real_value_df['route'] =='A_3']

real_value=list(real_value_df_A_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.21639528430495444

In [259]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[122.93, 157.342061640204],
 [125.95, 158.75426712730248],
 [125.28, 169.36695884097998],
 [132.03, 172.62471150613086],
 [123.7, 170.61324582554022],
 [225.87, 143.34947303824907],
 [115.72, 128.45690264152174],
 [134.77, 127.83704687309503],
 [124.86, 123.91385498893423],
 [110.21, 121.95510344917102],
 [94.86, 127.73147610955655],
 [130.83, 127.75392504239767],
 [120.8, 135.58897519475042],
 [115.92, 136.8059383688948],
 [156.26, 145.9513885969835],
 [127.25, 159.81419886847166],
 [131.14, 157.95200443819394],
 [118.32, 132.71148140926346],
 [106.79, 124.59695666927654],
 [128.15, 123.99572667904867],
 [101.61, 120.19042109293969],
 [112.62, 118.29052723198599],
 [142.51, 123.89332816577569],
 [96.71, 123.91510254033211],
 [127.49, 135.58897519475042],
 [107.83, 136.8059383688948],
 [104.91, 145.9513885969835],
 [272.65, 159.81419886847166],
 [227.0, 157.95200443819394],
 [101.48, 132.71148140926346],
 [115.54, 125.34798959178906],
 [93.73, 124.74313556828838],
 [108.61, 120.914892

## B3

In [260]:
#B3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_3_travel_times=travel_times['B_3']
B_3_travel_times_need={}
for i in B_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_3_travel_times_need:
            B_3_travel_times_need[i]=[]
        if i in B_3_travel_times_need:
            B_3_travel_times_need[i].extend(B_3_travel_times[i])
        
B_3_ratio_8_9={}
B_3_ratio_9_10={}
B_3_ratio_17_18={}
B_3_ratio_18_19={}

#date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_3_ratio_8_9[i]={}
    B_3_ratio_9_10[i]={}
    B_3_ratio_17_18[i]={}
    B_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_3_ratio_18_19[i][j]=[]

for i in B_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    for k in [480,500,520]:
        if B_3_ratio_8_9[i][k]==[] and B_3_ratio_8_9[i][k-20]!=[] and B_3_ratio_8_9[i][k+20]!=[]:
            B_3_ratio_8_9[i][k]=[np.mean([np.mean(B_3_ratio_8_9[i][k-20]),np.mean(B_3_ratio_8_9[i][k+20])])]
            
for i in B_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_3_ratio_9_10[i][k]==[] and B_3_ratio_9_10[i][k-20]!=[] and B_3_ratio_9_10[i][k+20]!=[]:
            B_3_ratio_9_10[i][k]=[np.mean([np.mean(B_3_ratio_9_10[i][k-20]),np.mean(B_3_ratio_9_10[i][k+20])])]
            
for i in B_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_3_ratio_17_18[i][k]==[] and B_3_ratio_17_18[i][k-20]!=[] and B_3_ratio_17_18[i][k+20]!=[]:
            B_3_ratio_17_18[i][k]=[np.mean([np.mean(B_3_ratio_17_18[i][k-20]),np.mean(B_3_ratio_17_18[i][k+20])])]
            
for i in B_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_3_ratio_18_19[i][k]==[] and B_3_ratio_18_19[i][k-20]!=[] and B_3_ratio_18_19[i][k+20]!=[]:
            B_3_ratio_18_19[i][k]=[np.mean([np.mean(B_3_ratio_18_19[i][k-20]),np.mean(B_3_ratio_18_19[i][k+20])])]
            
B_3_ratio_8_9_key=list(B_3_ratio_8_9.keys())
for i in B_3_ratio_8_9_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_3_ratio_8_9[i]
        
B_3_ratio_9_10_key=list(B_3_ratio_9_10.keys())
for i in B_3_ratio_9_10_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_3_ratio_9_10[i]

B_3_ratio_17_18_key=list(B_3_ratio_17_18.keys())
for i in B_3_ratio_17_18_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_3_ratio_17_18[i]

B_3_ratio_18_19_key=list(B_3_ratio_18_19.keys())
for i in  B_3_ratio_18_19_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    del B_3_ratio_8_9[i][8*60-20]
    del B_3_ratio_8_9[i][9*60]
    
for i in B_3_ratio_9_10:
    del B_3_ratio_9_10[i][9*60-20]
    del B_3_ratio_9_10[i][10*60]
    
for i in B_3_ratio_17_18:
    del B_3_ratio_17_18[i][17*60-20]
    del B_3_ratio_17_18[i][18*60]
    
for i in B_3_ratio_18_19:
    del B_3_ratio_18_19[i][18*60-20]
    del B_3_ratio_18_19[i][19*60]
    
for i in B_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_3_ratio_8_9[i]: 
        if B_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_8_9[i][j]))
            temp2.append(np.mean(B_3_ratio_8_9[i][j]))
        if B_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_3_ratio_9_10[i]: 
        if B_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_9_10[i][j]))
            temp2.append(np.mean(B_3_ratio_9_10[i][j]))
        if B_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_3_ratio_17_18[i]: 
        if B_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_17_18[i][j]))
            temp2.append(np.mean(B_3_ratio_17_18[i][j]))
        if B_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_3_ratio_18_19[i]: 
        if B_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_18_19[i][j]))
            temp2.append(np.mean(B_3_ratio_18_19[i][j]))
        if B_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_3_ratio_8_9_480=[]
B_3_ratio_8_9_500=[]
B_3_ratio_8_9_520=[]

for i in B_3_ratio_8_9:
    for j in B_3_ratio_8_9[i]:
        if j==480: B_3_ratio_8_9_480.extend(B_3_ratio_8_9[i][j])
        if j==500: B_3_ratio_8_9_500.extend(B_3_ratio_8_9[i][j])
        if j==520: B_3_ratio_8_9_520.extend(B_3_ratio_8_9[i][j])
    
    
B_3_ratio_9_10_540=[]
B_3_ratio_9_10_560=[]
B_3_ratio_9_10_580=[]

for i in B_3_ratio_9_10:
    for j in B_3_ratio_9_10[i]:
        if j==540: B_3_ratio_9_10_540.extend(B_3_ratio_9_10[i][j])
        if j==560: B_3_ratio_9_10_560.extend(B_3_ratio_9_10[i][j])
        if j==580: B_3_ratio_9_10_580.extend(B_3_ratio_9_10[i][j])
            
B_3_ratio_17_18_1020=[]
B_3_ratio_17_18_1040=[]
B_3_ratio_17_18_1060=[]

for i in B_3_ratio_17_18:
    for j in B_3_ratio_17_18[i]:
        if j==1020: B_3_ratio_17_18_1020.extend(B_3_ratio_17_18[i][j])
        if j==1040: B_3_ratio_17_18_1040.extend(B_3_ratio_17_18[i][j])
        if j==1060: B_3_ratio_17_18_1060.extend(B_3_ratio_17_18[i][j])
    
B_3_ratio_18_19_1080=[]
B_3_ratio_18_19_1100=[]
B_3_ratio_18_19_1120=[]

for i in B_3_ratio_18_19:
    for j in B_3_ratio_18_19[i]:
        if j==1080: B_3_ratio_18_19_1080.extend(B_3_ratio_18_19[i][j])
        if j==1100: B_3_ratio_18_19_1100.extend(B_3_ratio_18_19[i][j])
        if j==1120: B_3_ratio_18_19_1120.extend(B_3_ratio_18_19[i][j])
            
B_3_ratio_8_9_480.sort()
B_3_ratio_8_9_480_used=B_3_ratio_8_9_480[5:len(B_3_ratio_8_9_480)-5]

B_3_ratio_8_9_500.sort()
B_3_ratio_8_9_500_used=B_3_ratio_8_9_500[5:len(B_3_ratio_8_9_500)-5]

B_3_ratio_8_9_520.sort()
B_3_ratio_8_9_520_used=B_3_ratio_8_9_520[5:len(B_3_ratio_8_9_520)-5]

B_3_ratio_9_10_540.sort()
B_3_ratio_9_10_540_used=B_3_ratio_9_10_540[5:len(B_3_ratio_9_10_540)-5]

B_3_ratio_9_10_560.sort()
B_3_ratio_9_10_560_used=B_3_ratio_9_10_560[5:len(B_3_ratio_9_10_560)-5]

B_3_ratio_9_10_580.sort()
B_3_ratio_9_10_580_used=B_3_ratio_9_10_580[5:len(B_3_ratio_9_10_580)-5]

B_3_ratio_17_18_1020.sort()
B_3_ratio_17_18_1020_used=B_3_ratio_17_18_1020[5:len(B_3_ratio_17_18_1020)-5]

B_3_ratio_17_18_1040.sort()
B_3_ratio_17_18_1040_used=B_3_ratio_17_18_1040[5:len(B_3_ratio_17_18_1040)-5]

B_3_ratio_17_18_1060.sort()
B_3_ratio_17_18_1060_used=B_3_ratio_17_18_1060[5:len(B_3_ratio_17_18_1060)-5]

B_3_ratio_18_19_1080.sort()
B_3_ratio_18_19_1080_used=B_3_ratio_18_19_1080[5:len(B_3_ratio_18_19_1080)-5]

B_3_ratio_18_19_1100.sort()
B_3_ratio_18_19_1100_used=B_3_ratio_18_19_1100[5:len(B_3_ratio_18_19_1100)-5]

B_3_ratio_18_19_1120.sort()
B_3_ratio_18_19_1120_used=B_3_ratio_18_19_1120[5:len(B_3_ratio_18_19_1120)-5]

names=locals()
B_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_3_ratio.append(np.mean(names['B_3_ratio_'+str(i)+'_used']))
    
B_3_ratio_8=B_3_ratio[0:3]
B_3_ratio_9=B_3_ratio[3:6]
B_3_ratio_17=B_3_ratio[6:9]
B_3_ratio_18=B_3_ratio[9:12]




In [261]:
print(B_3_ratio_8)
print(B_3_ratio_9)
print(B_3_ratio_17)
print(B_3_ratio_18)

[0.9588587319159432, 0.9877176009992981, 1.0498548948656439]
[1.0254018000740626, 1.0251925941467648, 0.9276552578492043]
[0.9887454541008326, 0.9964707338380407, 1.0048398973940167]
[1.0134617557170276, 0.9490581825784564, 1.0192924016085947]


In [262]:
#B_3
file_path = '/Users/vayne/Desktop/validation/B_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(valid)

gdbt = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.10536970076864392
0.19815368489358295
          label           0
0    147.723333  132.493992
1    148.253333  139.486583
2    107.110000  109.996621
3    155.416667  142.867707
4    103.586667  126.568005
..          ...         ...
284  103.366667  113.617166
285   90.780000  101.728705
286  111.426667  113.324150
287  104.705000  102.887346
288  121.530000  102.584757

[289 rows x 2 columns]
         label           0
0   102.530000  102.107020
1   117.593333  101.825995
2   149.496667   84.515350
3   107.776667  102.846273
4   162.086667  113.205663
..         ...         ...
55  102.470000  108.406498
56  118.543333  106.866195
57   94.883333  122.162819
58  113.926667  105.748627
59   79.440000  135.703469

[60 rows x 2 columns]


In [264]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])


In [265]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_B_3.loc[real_value_df_B_3['route'] == 'B_3']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_B_3_time=list(real_value_df_B_3['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_B_3_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_B_3=real_value_df.loc[real_value_df['route'] =='B_3']

real_value=list(real_value_df_B_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.2036485483849047

## C_1

In [266]:
#C1

#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_1_travel_times=travel_times['C_1']
C_1_travel_times_need={}
for i in C_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_1_travel_times_need:
            C_1_travel_times_need[i]=[]
        if i in C_1_travel_times_need:
            C_1_travel_times_need[i].extend(C_1_travel_times[i])
        
C_1_ratio_8_9={}
C_1_ratio_9_10={}
C_1_ratio_17_18={}
C_1_ratio_18_19={}


date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_1_ratio_8_9[i]={}
    C_1_ratio_9_10[i]={}
    C_1_ratio_17_18[i]={}
    C_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_1_ratio_18_19[i][j]=[]

for i in C_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    for k in [480,500,520]:
        if C_1_ratio_8_9[i][k]==[] and C_1_ratio_8_9[i][k-20]!=[] and C_1_ratio_8_9[i][k+20]!=[]:
            C_1_ratio_8_9[i][k]=[np.mean([np.mean(C_1_ratio_8_9[i][k-20]),np.mean(C_1_ratio_8_9[i][k+20])])]
            
for i in C_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_1_ratio_9_10[i][k]==[] and C_1_ratio_9_10[i][k-20]!=[] and C_1_ratio_9_10[i][k+20]!=[]:
            C_1_ratio_9_10[i][k]=[np.mean([np.mean(C_1_ratio_9_10[i][k-20]),np.mean(C_1_ratio_9_10[i][k+20])])]
            
for i in C_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_1_ratio_17_18[i][k]==[] and C_1_ratio_17_18[i][k-20]!=[] and C_1_ratio_17_18[i][k+20]!=[]:
            C_1_ratio_17_18[i][k]=[np.mean([np.mean(C_1_ratio_17_18[i][k-20]),np.mean(C_1_ratio_17_18[i][k+20])])]
            
for i in C_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_1_ratio_18_19[i][k]==[] and C_1_ratio_18_19[i][k-20]!=[] and C_1_ratio_18_19[i][k+20]!=[]:
            C_1_ratio_18_19[i][k]=[np.mean([np.mean(C_1_ratio_18_19[i][k-20]),np.mean(C_1_ratio_18_19[i][k+20])])]
            
C_1_ratio_8_9_key=list(C_1_ratio_8_9.keys())
for i in C_1_ratio_8_9_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_1_ratio_8_9[i]
        
C_1_ratio_9_10_key=list(C_1_ratio_9_10.keys())
for i in C_1_ratio_9_10_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_1_ratio_9_10[i]

C_1_ratio_17_18_key=list(C_1_ratio_17_18.keys())
for i in C_1_ratio_17_18_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_1_ratio_17_18[i]

C_1_ratio_18_19_key=list(C_1_ratio_18_19.keys())
for i in  C_1_ratio_18_19_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    del C_1_ratio_8_9[i][8*60-20]
    del C_1_ratio_8_9[i][9*60]
    
for i in C_1_ratio_9_10:
    del C_1_ratio_9_10[i][9*60-20]
    del C_1_ratio_9_10[i][10*60]
    
for i in C_1_ratio_17_18:
    del C_1_ratio_17_18[i][17*60-20]
    del C_1_ratio_17_18[i][18*60]
    
for i in C_1_ratio_18_19:
    del C_1_ratio_18_19[i][18*60-20]
    del C_1_ratio_18_19[i][19*60]
    
for i in C_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_1_ratio_8_9[i]: 
        if C_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_8_9[i][j]))
            temp2.append(np.mean(C_1_ratio_8_9[i][j]))
        if C_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_1_ratio_9_10[i]: 
        if C_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_9_10[i][j]))
            temp2.append(np.mean(C_1_ratio_9_10[i][j]))
        if C_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_1_ratio_17_18[i]: 
        if C_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_17_18[i][j]))
            temp2.append(np.mean(C_1_ratio_17_18[i][j]))
        if C_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_1_ratio_18_19[i]: 
        if C_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_18_19[i][j]))
            temp2.append(np.mean(C_1_ratio_18_19[i][j]))
        if C_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_1_ratio_8_9_480=[]
C_1_ratio_8_9_500=[]
C_1_ratio_8_9_520=[]

for i in C_1_ratio_8_9:
    for j in C_1_ratio_8_9[i]:
        if j==480: C_1_ratio_8_9_480.extend(C_1_ratio_8_9[i][j])
        if j==500: C_1_ratio_8_9_500.extend(C_1_ratio_8_9[i][j])
        if j==520: C_1_ratio_8_9_520.extend(C_1_ratio_8_9[i][j])
    
    
C_1_ratio_9_10_540=[]
C_1_ratio_9_10_560=[]
C_1_ratio_9_10_580=[]

for i in C_1_ratio_9_10:
    for j in C_1_ratio_9_10[i]:
        if j==540: C_1_ratio_9_10_540.extend(C_1_ratio_9_10[i][j])
        if j==560: C_1_ratio_9_10_560.extend(C_1_ratio_9_10[i][j])
        if j==580: C_1_ratio_9_10_580.extend(C_1_ratio_9_10[i][j])
            
C_1_ratio_17_18_1020=[]
C_1_ratio_17_18_1040=[]
C_1_ratio_17_18_1060=[]

for i in C_1_ratio_17_18:
    for j in C_1_ratio_17_18[i]:
        if j==1020: C_1_ratio_17_18_1020.extend(C_1_ratio_17_18[i][j])
        if j==1040: C_1_ratio_17_18_1040.extend(C_1_ratio_17_18[i][j])
        if j==1060: C_1_ratio_17_18_1060.extend(C_1_ratio_17_18[i][j])
    
C_1_ratio_18_19_1080=[]
C_1_ratio_18_19_1100=[]
C_1_ratio_18_19_1120=[]

for i in C_1_ratio_18_19:
    for j in C_1_ratio_18_19[i]:
        if j==1080: C_1_ratio_18_19_1080.extend(C_1_ratio_18_19[i][j])
        if j==1100: C_1_ratio_18_19_1100.extend(C_1_ratio_18_19[i][j])
        if j==1120: C_1_ratio_18_19_1120.extend(C_1_ratio_18_19[i][j])
            
C_1_ratio_8_9_480.sort()
C_1_ratio_8_9_480_used=C_1_ratio_8_9_480[5:len(C_1_ratio_8_9_480)-5]

C_1_ratio_8_9_500.sort()
C_1_ratio_8_9_500_used=C_1_ratio_8_9_500[5:len(C_1_ratio_8_9_500)-5]

C_1_ratio_8_9_520.sort()
C_1_ratio_8_9_520_used=C_1_ratio_8_9_520[5:len(C_1_ratio_8_9_520)-5]

C_1_ratio_9_10_540.sort()
C_1_ratio_9_10_540_used=C_1_ratio_9_10_540[5:len(C_1_ratio_9_10_540)-5]

C_1_ratio_9_10_560.sort()
C_1_ratio_9_10_560_used=C_1_ratio_9_10_560[5:len(C_1_ratio_9_10_560)-5]

C_1_ratio_9_10_580.sort()
C_1_ratio_9_10_580_used=C_1_ratio_9_10_580[5:len(C_1_ratio_9_10_580)-5]

C_1_ratio_17_18_1020.sort()
C_1_ratio_17_18_1020_used=C_1_ratio_17_18_1020[5:len(C_1_ratio_17_18_1020)-5]

C_1_ratio_17_18_1040.sort()
C_1_ratio_17_18_1040_used=C_1_ratio_17_18_1040[5:len(C_1_ratio_17_18_1040)-5]

C_1_ratio_17_18_1060.sort()
C_1_ratio_17_18_1060_used=C_1_ratio_17_18_1060[5:len(C_1_ratio_17_18_1060)-5]

C_1_ratio_18_19_1080.sort()
C_1_ratio_18_19_1080_used=C_1_ratio_18_19_1080[5:len(C_1_ratio_18_19_1080)-5]

C_1_ratio_18_19_1100.sort()
C_1_ratio_18_19_1100_used=C_1_ratio_18_19_1100[5:len(C_1_ratio_18_19_1100)-5]

C_1_ratio_18_19_1120.sort()
C_1_ratio_18_19_1120_used=C_1_ratio_18_19_1120[5:len(C_1_ratio_18_19_1120)-5]

names=locals()
C_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_1_ratio.append(np.mean(names['C_1_ratio_'+str(i)+'_used']))
    
C_1_ratio_8=C_1_ratio[0:3]
C_1_ratio_9=C_1_ratio[3:6]
C_1_ratio_17=C_1_ratio[6:9]
C_1_ratio_18=C_1_ratio[9:12]



In [267]:
print(C_1_ratio_8)
print(C_1_ratio_9)
print(C_1_ratio_17)
print(C_1_ratio_18)

[1.0085446631871167, 1.0135826326873116, 0.9693400182600777]
[0.9649815049634551, 1.0068986583841009, 1.0131470826840883]
[0.9455038549039972, 0.9966929710379031, 1.0406337231620946]
[0.95167652707838, 1.0515667415053014, 0.9895062988612441]


In [268]:
#C_1
file_path = '/Users/vayne/Desktop/validation/C_1.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('/Users/vayne/Desktop/validation/5min_shifted_C_1.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/validation/-5min_shifted_C_1.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=4, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.10830155005055248
0.1390390318323988
          label           0
0    140.245000  165.550055
1    231.923333  203.167686
2    146.070000  167.551726
3    190.573333  163.689319
4    207.370000  202.490334
..          ...         ...
426  252.450000  235.299272
427  268.755000  227.762922
428  249.116667  270.656296
429  272.230000  207.973517
430  238.060000  231.659337

[431 rows x 2 columns]
         label           0
0   159.553333  170.674003
1   154.173333  180.682035
2   174.570000  177.866385
3   147.140000  170.335084
4   149.433333  175.918158
..         ...         ...
53  282.260000  198.851887
54  230.510000  221.262081
55  198.123333  218.834351
56  266.680000  253.093154
57  267.110000  185.878092

[58 rows x 2 columns]


In [269]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])

In [270]:
test_compare

,label,0
0,159.553333,170.674003
1,154.173333,180.682035
2,174.570000,177.866385
3,147.140000,170.335084
4,149.433333,175.918158
...,...,...
53,282.260000,198.851887
54,230.510000,221.262081
55,198.123333,218.834351
56,266.680000,253.093154


In [271]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_C_1.loc[real_value_df_C_1['route'] == 'C_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_1_time=list(real_value_df_C_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_1_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_C_1=real_value_df.loc[real_value_df['route'] =='C_1']

real_value=list(real_value_df_C_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.19605690964044217

## C3

In [272]:

#find the travel time at each time window
file_path='/Users/vayne/Desktop/validation/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_3_travel_times=travel_times['C_3']
C_3_travel_times_need={}
for i in C_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_3_travel_times_need:
            C_3_travel_times_need[i]=[]
        if i in C_3_travel_times_need:
            C_3_travel_times_need[i].extend(C_3_travel_times[i])
        
C_3_ratio_8_9={}
C_3_ratio_9_10={}
C_3_ratio_17_18={}
C_3_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_3_ratio_8_9[i]={}
    C_3_ratio_9_10[i]={}
    C_3_ratio_17_18[i]={}
    C_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_3_ratio_18_19[i][j]=[]

for i in C_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    for k in [480,500,520]:
        if C_3_ratio_8_9[i][k]==[] and C_3_ratio_8_9[i][k-20]!=[] and C_3_ratio_8_9[i][k+20]!=[]:
            C_3_ratio_8_9[i][k]=[np.mean([np.mean(C_3_ratio_8_9[i][k-20]),np.mean(C_3_ratio_8_9[i][k+20])])]
            
for i in C_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_3_ratio_9_10[i][k]==[] and C_3_ratio_9_10[i][k-20]!=[] and C_3_ratio_9_10[i][k+20]!=[]:
            C_3_ratio_9_10[i][k]=[np.mean([np.mean(C_3_ratio_9_10[i][k-20]),np.mean(C_3_ratio_9_10[i][k+20])])]
            
for i in C_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_3_ratio_17_18[i][k]==[] and C_3_ratio_17_18[i][k-20]!=[] and C_3_ratio_17_18[i][k+20]!=[]:
            C_3_ratio_17_18[i][k]=[np.mean([np.mean(C_3_ratio_17_18[i][k-20]),np.mean(C_3_ratio_17_18[i][k+20])])]
            
for i in C_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_3_ratio_18_19[i][k]==[] and C_3_ratio_18_19[i][k-20]!=[] and C_3_ratio_18_19[i][k+20]!=[]:
            C_3_ratio_18_19[i][k]=[np.mean([np.mean(C_3_ratio_18_19[i][k-20]),np.mean(C_3_ratio_18_19[i][k+20])])]
            
C_3_ratio_8_9_key=list(C_3_ratio_8_9.keys())
for i in C_3_ratio_8_9_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_3_ratio_8_9[i]
        
C_3_ratio_9_10_key=list(C_3_ratio_9_10.keys())
for i in C_3_ratio_9_10_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_3_ratio_9_10[i]

C_3_ratio_17_18_key=list(C_3_ratio_17_18.keys())
for i in C_3_ratio_17_18_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_3_ratio_17_18[i]

C_3_ratio_18_19_key=list(C_3_ratio_18_19.keys())
for i in  C_3_ratio_18_19_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    del C_3_ratio_8_9[i][8*60-20]
    del C_3_ratio_8_9[i][9*60]
    
for i in C_3_ratio_9_10:
    del C_3_ratio_9_10[i][9*60-20]
    del C_3_ratio_9_10[i][10*60]
    
for i in C_3_ratio_17_18:
    del C_3_ratio_17_18[i][17*60-20]
    del C_3_ratio_17_18[i][18*60]
    
for i in C_3_ratio_18_19:
    del C_3_ratio_18_19[i][18*60-20]
    del C_3_ratio_18_19[i][19*60]
    
for i in C_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_3_ratio_8_9[i]: 
        if C_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_8_9[i][j]))
            temp2.append(np.mean(C_3_ratio_8_9[i][j]))
        if C_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_3_ratio_9_10[i]: 
        if C_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_9_10[i][j]))
            temp2.append(np.mean(C_3_ratio_9_10[i][j]))
        if C_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_3_ratio_17_18[i]: 
        if C_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_17_18[i][j]))
            temp2.append(np.mean(C_3_ratio_17_18[i][j]))
        if C_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_3_ratio_18_19[i]: 
        if C_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_18_19[i][j]))
            temp2.append(np.mean(C_3_ratio_18_19[i][j]))
        if C_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_3_ratio_8_9_480=[]
C_3_ratio_8_9_500=[]
C_3_ratio_8_9_520=[]

for i in C_3_ratio_8_9:
    for j in C_3_ratio_8_9[i]:
        if j==480: C_3_ratio_8_9_480.extend(C_3_ratio_8_9[i][j])
        if j==500: C_3_ratio_8_9_500.extend(C_3_ratio_8_9[i][j])
        if j==520: C_3_ratio_8_9_520.extend(C_3_ratio_8_9[i][j])
    
    
C_3_ratio_9_10_540=[]
C_3_ratio_9_10_560=[]
C_3_ratio_9_10_580=[]

for i in C_3_ratio_9_10:
    for j in C_3_ratio_9_10[i]:
        if j==540: C_3_ratio_9_10_540.extend(C_3_ratio_9_10[i][j])
        if j==560: C_3_ratio_9_10_560.extend(C_3_ratio_9_10[i][j])
        if j==580: C_3_ratio_9_10_580.extend(C_3_ratio_9_10[i][j])
            
C_3_ratio_17_18_1020=[]
C_3_ratio_17_18_1040=[]
C_3_ratio_17_18_1060=[]

for i in C_3_ratio_17_18:
    for j in C_3_ratio_17_18[i]:
        if j==1020: C_3_ratio_17_18_1020.extend(C_3_ratio_17_18[i][j])
        if j==1040: C_3_ratio_17_18_1040.extend(C_3_ratio_17_18[i][j])
        if j==1060: C_3_ratio_17_18_1060.extend(C_3_ratio_17_18[i][j])
    
C_3_ratio_18_19_1080=[]
C_3_ratio_18_19_1100=[]
C_3_ratio_18_19_1120=[]

for i in C_3_ratio_18_19:
    for j in C_3_ratio_18_19[i]:
        if j==1080: C_3_ratio_18_19_1080.extend(C_3_ratio_18_19[i][j])
        if j==1100: C_3_ratio_18_19_1100.extend(C_3_ratio_18_19[i][j])
        if j==1120: C_3_ratio_18_19_1120.extend(C_3_ratio_18_19[i][j])
            
C_3_ratio_8_9_480.sort()
C_3_ratio_8_9_480_used=C_3_ratio_8_9_480[5:len(C_3_ratio_8_9_480)-5]

C_3_ratio_8_9_500.sort()
C_3_ratio_8_9_500_used=C_3_ratio_8_9_500[5:len(C_3_ratio_8_9_500)-5]

C_3_ratio_8_9_520.sort()
C_3_ratio_8_9_520_used=C_3_ratio_8_9_520[5:len(C_3_ratio_8_9_520)-5]

C_3_ratio_9_10_540.sort()
C_3_ratio_9_10_540_used=C_3_ratio_9_10_540[5:len(C_3_ratio_9_10_540)-5]

C_3_ratio_9_10_560.sort()
C_3_ratio_9_10_560_used=C_3_ratio_9_10_560[5:len(C_3_ratio_9_10_560)-5]

C_3_ratio_9_10_580.sort()
C_3_ratio_9_10_580_used=C_3_ratio_9_10_580[5:len(C_3_ratio_9_10_580)-5]

C_3_ratio_17_18_1020.sort()
C_3_ratio_17_18_1020_used=C_3_ratio_17_18_1020[5:len(C_3_ratio_17_18_1020)-5]

C_3_ratio_17_18_1040.sort()
C_3_ratio_17_18_1040_used=C_3_ratio_17_18_1040[5:len(C_3_ratio_17_18_1040)-5]

C_3_ratio_17_18_1060.sort()
C_3_ratio_17_18_1060_used=C_3_ratio_17_18_1060[5:len(C_3_ratio_17_18_1060)-5]

C_3_ratio_18_19_1080.sort()
C_3_ratio_18_19_1080_used=C_3_ratio_18_19_1080[5:len(C_3_ratio_18_19_1080)-5]

C_3_ratio_18_19_1100.sort()
C_3_ratio_18_19_1100_used=C_3_ratio_18_19_1100[5:len(C_3_ratio_18_19_1100)-5]

C_3_ratio_18_19_1120.sort()
C_3_ratio_18_19_1120_used=C_3_ratio_18_19_1120[5:len(C_3_ratio_18_19_1120)-5]

names=locals()
C_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_3_ratio.append(np.mean(names['C_3_ratio_'+str(i)+'_used']))
    
C_3_ratio_8=C_3_ratio[0:3]
C_3_ratio_9=C_3_ratio[3:6]
C_3_ratio_17=C_3_ratio[6:9]
C_3_ratio_18=C_3_ratio[9:12]

In [273]:
print(C_3_ratio_8)
print(C_3_ratio_9)
print(C_3_ratio_17)
print(C_3_ratio_18)

[0.9148328918283646, 1.0075582153926481, 1.0225619584493677]
[1.040282893182057, 0.9801969562756571, 0.9588049191875893]
[0.9750481330349163, 0.9561918653994332, 1.064546493530613]
[1.0017435848975271, 0.980500959518688, 0.9999905668560394]


In [274]:
#C_3
file_path = '/Users/vayne/Desktop/validation/C_3.csv'

data_whole = pd.read_csv(file_path)
cal_day=list(data_whole['calculated_day'])

valid_date=[298,299,300,301,302,303,304,320,321,322,323,324,325,326,327]
valid_index=[]
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)
        
        
valid=data_whole.iloc[valid_index]
train=data_whole.drop(valid_index)
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)


train_p5= pd.read_csv('/Users/vayne/Desktop/validation/5min_shifted_C_3.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/validation/-5min_shifted_C_3.csv')


valid_index=[]

cal_day=list(train_p5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_p5_used=train_p5.drop(valid_index)
train_p5_used.reset_index(drop=True, inplace=True)


valid_index=[]

cal_day=list(train_m5['calculated_day'])
for i in range(len(cal_day)):
    if cal_day[i] in valid_date:
        valid_index.append(i)

train_m5_used=train_m5.drop(valid_index)
train_m5_used.reset_index(drop=True, inplace=True)


train1 = pd.concat([train,train_p5_used])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(valid)



gdbt = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, 
                                 max_depth=6, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.09362078428831462
0.19348686712619714
          label           0
0    132.050000  149.097780
1    219.150000  196.170221
2    166.150000  168.412379
3    161.786667  186.421348
4    207.950000  203.949970
..          ...         ...
393  307.935000  243.197690
394  189.050000  182.590378
395  245.666667  226.870641
396  141.100000  172.687845
397  182.425000  175.825383

[398 rows x 2 columns]
         label           0
0   198.080000  190.231380
1   276.740000  150.025399
2   288.013333  162.188556
3   210.320000  194.086580
4   311.870000  167.038322
..         ...         ...
52  178.660000  183.767163
53  152.910000  215.563308
54  250.595000  175.138302
55  142.770000  183.935643
56  350.115000  174.813660

[57 rows x 2 columns]


In [275]:
test=valid
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])


In [276]:
final_result.sort(key = lambda x: x[0]) 
real_value_df_C_3.loc[real_value_df_C_3['route'] == 'C_3']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_3_time=list(real_value_df_C_3['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_3_time:final_result3.append(i[0])
        
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    
#load the real value 
real_value_df_C_3=real_value_df.loc[real_value_df['route'] =='C_3']

real_value=list(real_value_df_C_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))
    
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.2371237212020904

In [277]:
summ=0.14426880667049694+0.18032249079453194+0.21639528430495444+0.2036485483849047+0.19605690964044217+0.2371237212020904

In [278]:
summ/6

0.19630262683290345